In [1]:
import os
import sys
sys.path.append('/home/haoc/TorchSSL_Benchmark/TorchSSL_Benchmark')

import numpy as np 
import torch
import torchvision
from torchvision import transforms

from src import get_dataset, get_data_loader, split_ssl_data, net_builder, get_algorithm, get_config, BasicDataset, Trainer

In [2]:
# first define configs and create config
config = {
    'algorithm': 'fixmatch',
    'net': 'wrn_28_2',
    'use_pretrain': True,

    'epoch': 10,
    'num_train_iter': 50000,
    'num_eval_iter': 5000,

    'optim': 'SGD',
    'lr': 0.03,
    'momentum': 0.9,

    'batch_size': 64,
    'dataset': 'cifar10',
    'num_classes': 10,
    'input_size': 32,

    'hard_label': True,
    'uratio': 3,
    'ulb_loss_ratio': 1.0,

    'gpu': 0,
    'world_size': 1,
    'distributed': False,
}   

config = get_config(config)
print(config)


Namespace(save_dir='./saved_models', save_name='fixmatch_cifar10', resume=False, load_path=None, overwrite=False, use_tensorboard=False, epoch=10, num_train_iter=50000, num_eval_iter=5000, num_labels=40, batch_size=64, uratio=3, eval_batch_size=1024, ema_m=0.999, ulb_loss_ratio=1.0, optim='SGD', lr=0.03, momentum=0.9, weight_decay=0.0005, net='wrn_28_2', net_from_name=False, use_pretrain=True, algorithm='fixmatch', use_cat=True, use_amp=False, clip_grad=0, data_dir='./data', dataset='cifar10', train_sampler='RandomSampler', num_classes=10, num_workers=1, max_length=512, max_length_seconds=4.0, sample_rate=16000, world_size=1, rank=0, dist_url='tcp://127.0.0.1:29300', dist_backend='nccl', seed=1, gpu=0, multiprocessing_distributed=False, hard_label=True, T=0.5, p_cutoff=0.95, input_size=32, distributed=False)


In [3]:
# create model, algorithm, trainer
algorithm = get_algorithm(config,  net_builder(config.net, from_name=False), tb_log=None, logger=None)

trainer = Trainer(config, algorithm)

_IncompatibleKeys(missing_keys=['fc.weight', 'fc.bias'], unexpected_keys=[])
_IncompatibleKeys(missing_keys=['fc.weight', 'fc.bias'], unexpected_keys=[])


In [4]:
# create dataset

# load labeled data and unlabeled data
lb_data, lb_target = None, None
ulb_data = None

# if no labeled data and unlabeled data specified, use ssl_split_data
data_dir = os.path.join(config.data_dir, config.dataset.lower())
dataset = getattr(torchvision.datasets, config.dataset.upper())(data_dir, train=True, download=True)
data, target = dataset.data, dataset.targets
lb_data, lb_target, ulb_data, ulb_target = split_ssl_data(config, data, target, num_labels=20, num_classes=config.num_classes)

train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.RandomCrop(32, padding=int(32 * 0.125), padding_mode='reflect'),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])


# weak augment
lb_dataset = BasicDataset(config.algorithm, lb_data, lb_target, config.num_classes, train_transform, is_ulb=False)
ulb_dataset = BasicDataset(config.algorithm, lb_data, lb_target, config.num_classes, train_transform, is_ulb=True)



# load test data
eval_data, eval_target = None, None
dataset = getattr(torchvision.datasets, config.dataset.upper())(data_dir, train=False, download=True)
eval_data, eval_target = dataset.data, dataset.targets


eval_transform =  transforms.Compose([transforms.Resize(32),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

eval_dataset = BasicDataset(config.algorithm, lb_data, lb_target, config.num_classes, eval_transform, is_ulb=False)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# get data loaders
train_lb_loader = get_data_loader(config, lb_dataset, config.batch_size, num_workers=4)
train_ulb_loader = get_data_loader(config, ulb_dataset, int(config.batch_size * config.uratio), num_workers=8)
eval_loader = get_data_loader(config, eval_dataset, config.eval_batch_size, num_workers=4)
print(len(train_lb_loader))

5000


In [6]:
# train
trainer.fit(train_lb_loader, train_ulb_loader, eval_loader)


104it [00:33,  3.13it/s]| 104/5000 [00:34<24:00,  3.40it/s, train/mask_ratio=1, train/sup_loss=2.15, train/total_loss=2.15, train/unsup_loss=0]


KeyboardInterrupt: 

In [7]:

                                      
# evaluate
trainer.evaluate(eval_loader)

UnboundLocalError: local variable 'total_num' referenced before assignment

In [ ]:

# predict
y_pred, y_logits = trainer.predict(eval_loader)
